# Pandemic Preparedness Analysis - Vaccine Administration Table

In [1]:
#Dependencies
import pandas as pd
import os

In [2]:
# pulling in datasets

#cdc csv
cdc = "cdc_source.csv"

#google data
vaccine = "https://storage.googleapis.com/covid19-open-data/v3/vaccinations.csv"

In [3]:
# read in files
cdc_df = pd.read_csv(cdc)
vax_df = pd.read_csv(vaccine)

In [4]:
# keep: date, loc-key, new person vax, cumulative person vax, new persons full vax, \
# cumulative person full vax, new vax doses admined, cumul vax doses admin, 


vax_df = vax_df.drop(columns=['new_persons_vaccinated_pfizer', \
                               'cumulative_persons_vaccinated_pfizer', \
                               'new_persons_fully_vaccinated_pfizer', \
                               'cumulative_persons_fully_vaccinated_pfizer', \
                               'new_vaccine_doses_administered_pfizer', \
                               'cumulative_vaccine_doses_administered_pfizer', \
                               'new_persons_vaccinated_moderna', \
                               'cumulative_persons_vaccinated_moderna', \
                               'new_persons_fully_vaccinated_moderna', \
                               'cumulative_persons_fully_vaccinated_moderna', \
                               'new_vaccine_doses_administered_moderna', \
                               'cumulative_vaccine_doses_administered_moderna', \
                               'new_persons_vaccinated_janssen', \
                               'cumulative_persons_vaccinated_janssen', \
                               'new_persons_fully_vaccinated_janssen', \
                               'cumulative_persons_fully_vaccinated_janssen', \
                               'new_vaccine_doses_administered_janssen', \
                               'cumulative_vaccine_doses_administered_janssen', \
                               'new_persons_vaccinated_sinovac', \
                               'total_persons_vaccinated_sinovac', \
                               'new_persons_fully_vaccinated_sinovac', \
                               'total_persons_fully_vaccinated_sinovac', \
                               'new_vaccine_doses_administered_sinovac', \
                               'total_vaccine_doses_administered_sinovac'])


In [5]:
# clean vax columns

vax_df.rename(columns={'date':'Date',
 'location_key':'State',
 'new_persons_vaccinated':'New Persons Vaccinated',
 'cumulative_persons_vaccinated':'Cumulative Persons Vaccinated',
 'new_persons_fully_vaccinated':'New Persons Fully Vaccinated',
 'cumulative_persons_fully_vaccinated':'Cumulative Persons Fully Vaccinated',
 'new_vaccine_doses_administered':'New Vaccines Doses Admined',
 'cumulative_vaccine_doses_administered':'Cumulative Vaccines Doses Admined'}, inplace=True)

In [6]:
#cdc column cleanup

cdc_df.rename(columns={'submission_date':'Date','state':'State', 'tot_cases':'Total Cases', \
'new_case': 'Confirmed New C19 Case', 'tot_death': 'Total Death', \
                      'new_death': 'Confirmed New C19 Death'}, inplace=True)

In [7]:
vax_df['State'] = vax_df['State'].replace({'US_NY':'NY','US_TX':'TX'}) 

In [8]:
cdc_df = cdc_df.astype({'Date':"datetime64"})

In [9]:
vax_df = vax_df.astype({'Date':"datetime64"})

In [12]:
# filter out tables on states and dates

cdc_df = pd.DataFrame(cdc_df.loc[(cdc_df['State'] == 'NY') | (cdc_df['State'] == 'TX'), :])

In [14]:
vax_df = pd.DataFrame(vax_df.loc[(vax_df['State'] == 'NY') | (vax_df['State'] == 'TX'), :])

In [16]:
#filter down dates

#cdc October 2020 to Dec 2021
cdc_post = pd.DataFrame(cdc_df.loc[(cdc_df['Date'] > '2020-09-30') & (cdc_df['Date'] \
                                                                      <'2022-01-01'), :])

In [21]:
#vax October 2020 to Dec 2021

vax_df = pd.DataFrame(vax_df.loc[(vax_df['Date'] > '2020-09-30') & (vax_df['Date'] \
                                                                    <'2022-01-01'), :])

In [20]:
cdc_post.count()

Date                       914
State                      914
Total Cases                914
Confirmed New C19 Case     914
Total Death                914
Confirmed New C19 Death    914
dtype: int64

In [22]:
vax_df.count()

Date                                   702
State                                  702
New Persons Vaccinated                 697
Cumulative Persons Vaccinated          700
New Persons Fully Vaccinated           696
Cumulative Persons Fully Vaccinated    696
New Vaccines Doses Admined             700
Cumulative Vaccines Doses Admined      702
dtype: int64

In [23]:
cdc_post.to_csv("cdc_post_vax.csv", index=False, header=True)

In [27]:
#merge tables keep nan

vax_stats = pd.merge(cdc_post, vax_df, how = 'left', on = ['Date','State'])

In [28]:
vax_stats.count()

Date                                   914
State                                  914
Total Cases                            914
Confirmed New C19 Case                 914
Total Death                            914
Confirmed New C19 Death                914
New Persons Vaccinated                 697
Cumulative Persons Vaccinated          700
New Persons Fully Vaccinated           696
Cumulative Persons Fully Vaccinated    696
New Vaccines Doses Admined             700
Cumulative Vaccines Doses Admined      702
dtype: int64

In [30]:
vax_stats = vax_stats.sort_values(by=['State','Date'], ascending = [True, True])

In [32]:
vax_stats = vax_stats.reset_index(drop=True)

In [33]:
vax_stats

Date State Total Cases Confirmed New C19 Case Total Death  \
0   2020-10-01    NY     215,457                    849       9,046   
1   2020-10-02    NY     216,456                    999       9,050   
2   2020-10-03    NY     217,475                  1,019       9,057   
3   2020-10-04    NY     218,165                    690       9,068   
4   2020-10-05    NY     218,630                    465       9,073   
..         ...   ...         ...                    ...         ...   
909 2021-12-27    TX   4,516,356                 17,331      74,262   
910 2021-12-28    TX   4,531,151                 14,795      74,330   
911 2021-12-29    TX   4,545,437                 14,286      74,401   
912 2021-12-30    TX   4,561,037                 15,600      74,491   
913 2021-12-31    TX   4,576,799                 15,762      74,491   

    Confirmed New C19 Death  New Persons Vaccinated  \
0                         5                     NaN   
1                         4                     NaN   
2                         7                     NaN   
3                        11                     NaN   
4                         5                     NaN   
..                      ...                     ...   
909                     170                 60469.0   
910                      68                 21531.0   
911                      71                 32800.0   
912                      90                 35165.0   
913                       0                     0.0   

     Cumulative Persons Vaccinated  New Persons Fully Vaccinated  \
0                              NaN                           NaN   
1                              NaN                           NaN   
2                              NaN                           NaN   
3                              NaN                           NaN   
4                              NaN                           NaN   
..                             ...                           ...   
909                     19282639.0                       39565.0   
910                     19304170.0                       14283.0   
911                     19336970.0                       20321.0   
912                     19372135.0                       19848.0   
913                     19372135.0                           0.0   

     Cumulative Persons Fully Vaccinated  New Vaccines Doses Admined  \
0                                    NaN                         NaN   
1                                    NaN                         NaN   
2                                    NaN                         NaN   
3                                    NaN                         NaN   
4                                    NaN                         NaN   
..                                   ...                         ...   
909                           16462911.0                    256062.0   
910                           16477194.0                     80208.0   
911                           16497515.0                    128149.0   
912                           16517363.0                    129039.0   
913                           16517363.0                         0.0   

     Cumulative Vaccines Doses Admined  
0                                  NaN  
1                                  NaN  
2                                  NaN  
3                                  NaN  
4                                  NaN  
..                                 ...  
909                         40274677.0  
910                         40354885.0  
911                         40483034.0  
912                         40612073.0  
913                         40612073.0  

[914 rows x 12 columns]

In [34]:
vax_stats.to_csv("vax_rates.csv", index=False, header=True)